In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import nept
import seaborn as sns

import scalebar

from loading_data import get_data
from plot_sequence_raster import plot_sequence
from run import spike_sorted_infos
from analyze_tuning_curves import get_tuning_curves
from utils_maze import speed_threshold
from analyze_decode_bytrial import get_trials

In [ ]:
from matplotlib import animation
from IPython.display import HTML

In [ ]:
thisdir = os.getcwd()
pickle_filepath = os.path.join(thisdir, "cache", "pickled")
output_filepath = os.path.join(thisdir, "plots", "exploring_swrs")

In [ ]:
import info.r068d5 as r068d5
infos = [r068d5]

In [ ]:
# infos = spike_sorted_infos

In [ ]:
def plot_swrs_stats(data, task_times, title, ylabel, savepath=None):
    fig, ax = plt.subplots()
    ind = np.arange(len(task_times))

    plt.bar(ind, data)

    ax.set_xticks(ind)
    ax.set_xticklabels(task_times, rotation=75, fontsize=14)
    
    plt.ylabel(ylabel)
    plt.title(title)
    
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    # plt.axhline(y=10, color='r', linestyle=':')
    plt.tight_layout()

    if savepath is not None:
        plt.savefig(savepath + ".png")
        plt.close("all")
    else:
        plt.show()

In [ ]:
def plot_swr(swrs, lfp, position, spikes, buffer=0.15, n_plots=10, savepath=None):
    if swrs.n_epochs < n_plots:
        starts = swrs.starts
        stops = swrs.stops
    else:
        starts = swrs.starts[:n_plots]
        stops = swrs.stops[:n_plots]
        
    for i, (start, stop) in enumerate(zip(starts, stops)):
        start_time = start - buffer
        stop_time = stop + buffer

        rows = len(spikes)
        add_rows = int(rows / 8)

        ms = 800 / rows
        mew = 0.7
        spike_loc = 1

        fig = plt.figure(figsize=(8, 8))
        ax1 = plt.subplot2grid((rows+add_rows, 2), (0, 0), rowspan=rows)

        for idx, neuron_spikes in enumerate(spikes):
            ax1.plot(neuron_spikes.time, np.ones(len(neuron_spikes.time))+(idx*spike_loc), '|',
                     color='k', ms=ms, mew=mew)
        ax1.set_xticks([])
        ax1.set_xlim([start_time, stop_time])
        ax1.set_ylim([0.5, len(spikes)*spike_loc+0.5])

        ax2 = plt.subplot2grid((rows+add_rows, 2), (rows, 0), rowspan=add_rows, sharex=ax1)
        start_idx = nept.find_nearest_idx(lfp.time, start_time)
        stop_idx = nept.find_nearest_idx(lfp.time, stop_time)
        ax2.plot(lfp.time[start_idx:stop_idx], lfp.data[start_idx:stop_idx], '#3288bd', lw=0.3)
        start_idx = nept.find_nearest_idx(lfp.time, start)
        stop_idx = nept.find_nearest_idx(lfp.time, stop)
        ax2.plot(lfp.time[start_idx:stop_idx], lfp.data[start_idx:stop_idx], 'r', lw=0.4)
        ax2.set_xticks([])
        ax2.set_xlim([start_time, stop_time])
        ax2.set_yticks([])

        scalebar.add_scalebar(ax2, matchy=False, bbox_transform=fig.transFigure,
                              bbox_to_anchor=(0.5, 0.05), units='ms')

        ax3 = plt.subplot2grid((rows+add_rows, 2), (0, 1), rowspan=int(0.5*rows))
        start_idx = nept.find_nearest_idx(position.time, start_time)
        stop_idx = nept.find_nearest_idx(position.time, stop_time)
        ax3.plot(position.x, position.y, '.', color="#bdbdbd", ms=2)
        ax3.plot(position.x[start_idx:stop_idx], position.y[start_idx:stop_idx], 'k.')
        ax3.set_xlim(np.min(position.x), np.max(position.x))
        ax3.set_ylim(np.min(position.y), np.max(position.y))
        ax3.axis('off')

        sns.despine(bottom=True)
        plt.tight_layout(h_pad=0.003)

        if savepath is not None:
            plt.savefig(savepath + str(i) + ".png", bbox_inches='tight')
            plt.close("all")
        else:
            plt.show()

In [ ]:

for info in infos:
    events, position, spikes, lfp, _ = get_data(info)
    
    trial_epochs = get_trials(events, info.task_times["phase3"])
    
    trial_idx = 10   
    trial_start = trial_epochs.starts[trial_idx]
    trial_stop = trial_epochs.stops[trial_idx]
    trial_times = nept.Epoch([trial_start, trial_stop])
    
    phase = info.task_times["phase1"]
    sliced_position = position.time_slice(trial_times.start, trial_times.stop)
    resting_epochs = speed_threshold(sliced_position, speed_limit=4., rest=True)
    rest_position = sliced_position[resting_epochs]

    sliced_lfp = lfp.time_slice(resting_epochs.starts, resting_epochs.stops)
    sliced_spikes = [spiketrain.time_slice(resting_epochs.starts, resting_epochs.stops) for spiketrain in spikes]

In [ ]:
plt.plot(sliced_position.time, sliced_position.y, "k.")
plt.plot(rest_position.time, rest_position.y, "b.")
plt.show()

In [ ]:
xedges, yedges = nept.get_xyedges(position)

fig, ax = plt.subplots()

xx, yy = np.meshgrid(xedges, yedges)

pad_amount = 5
ax.set_xlim((np.floor(np.min(rest_position.x))-pad_amount, np.ceil(np.max(rest_position.x))+pad_amount))
ax.set_ylim((np.floor(np.min(rest_position.y))-pad_amount, np.ceil(np.max(rest_position.y))+pad_amount))

plt.plot(sliced_position.x, sliced_position.y, '.', color="#bdbdbd")
rat_position, = ax.plot([], [], "<", color="b")

fig.tight_layout()


def init():
    rat_position.set_data([], [])
    return rat_position


def animate(i):
    rat_position.set_data(rest_position.x[i], rest_position.y[i])
    return rat_position

anim = animation.FuncAnimation(fig, animate, frames=rest_position.n_samples, interval=80, 
                               blit=False, repeat=False)

In [ ]:
HTML(anim.to_html5_video())

In [ ]:
info = r068d5
remove_interneurons = False
resting_only = True

print(info.session_id)
events, position, spikes, lfp, _ = get_data(info)

condition = ""

if remove_interneurons:
    max_mean_firing = 5
    interneurons = np.zeros(len(spikes), dtype=bool)
    for i, spike in enumerate(spikes):
        if len(spike.time) / info.session_length >= max_mean_firing:
            interneurons[i] = True
    spikes = spikes[~interneurons]
    condition = condition + "_no-interneurons"

# task_times = ["prerecord", "phase1", "pauseA", "phase2", "pauseB", "phase3", "postrecord"]
task_times = ["pauseB"]

In [ ]:
n_swrs = np.zeros(len(task_times))
phase_duration = np.zeros(len(task_times))

for i, task_time in enumerate(task_times):
    if remove_interneurons:
        condition = "_no-interneurons"
    else:
        condition = ""
        
    epochs_of_interest = info.task_times[task_time]

    phase_duration[i] = epochs_of_interest.durations[0] / 60.

    if resting_only:
        sliced_position = position.time_slice(epochs_of_interest.start, epochs_of_interest.stop)
        plt.plot(sliced_position.x, sliced_position.y, "k.")
        plt.show()
        epochs_of_interest = speed_threshold(sliced_position, speed_limit=4., rest=True)
        condition = condition + "_rest"
        print(condition)

    sliced_lfp = lfp.time_slice(epochs_of_interest.starts, epochs_of_interest.stops)
    sliced_spikes = [spiketrain.time_slice(epochs_of_interest.starts, epochs_of_interest.stops) for spiketrain in
                     spikes]

    z_thresh = 2.0
    power_thresh = 3.0
    merge_thresh = 0.02
    min_length = 0.05
    swrs = nept.detect_swr_hilbert(sliced_lfp, fs=info.fs, thresh=(140.0, 250.0), z_thresh=z_thresh,
                                   power_thresh=power_thresh, merge_thresh=merge_thresh, min_length=min_length)

    multi_swrs = nept.find_multi_in_epochs(sliced_spikes, swrs, min_involved=4)

    n_swrs[i] = multi_swrs.n_epochs

In [ ]:
print("n_swrs:", n_swrs)
print("swr_rate:", n_swrs / phase_duration)